# Read Data

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('SUI_USDT_futures_15m_20241231.csv')

# Drop unused columns
df.drop(columns=['coin','unused', 'closetime'], inplace=True)

# Display the first few rows of the DataFrame
df

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote
0,1683129600000,1.4030,1.4030,1.2652,1.3338,11474534.0,1.517217e+07,43648,5505355.7,7.277550e+06
1,1683130500000,1.3339,1.3863,1.3327,1.3716,16090260.4,2.197104e+07,72866,8103409.6,1.106807e+07
2,1683131400000,1.3717,1.3957,1.3547,1.3857,13192974.2,1.820509e+07,57805,6829156.6,9.423169e+06
3,1683132300000,1.3866,1.3924,1.3652,1.3733,11951258.0,1.646971e+07,46428,5673737.9,7.822083e+06
4,1683133200000,1.3731,1.3756,1.3405,1.3574,12698235.4,1.719730e+07,50735,6035706.0,8.176917e+06
...,...,...,...,...,...,...,...,...,...,...
58323,1735620300000,4.0787,4.0850,4.0616,4.0828,1863281.5,7.589595e+06,20948,927866.5,3.779383e+06
58324,1735621200000,4.0829,4.0953,4.0738,4.0810,854988.6,3.493640e+06,13288,392507.9,1.603318e+06
58325,1735622100000,4.0811,4.0990,4.0637,4.0982,1221874.8,4.985167e+06,18433,781802.8,3.190057e+06
58326,1735623000000,4.0982,4.1310,4.0900,4.1198,1702835.3,7.003625e+06,26801,850966.2,3.498465e+06


# Explore

In [41]:
df_setup = df.copy()

# Define bull/bear candle
df_setup['bull'] = (df_setup['closeprice'] > df_setup['openprice']).astype(int)

# Calculate candle body size
df_setup['body_size'] = abs(df_setup['closeprice'] - df_setup['openprice'])

# Calculate tail length
df_setup['high_tail'] = abs(df_setup['highprice'] - df_setup[['openprice', 'closeprice']].max(axis=1))
df_setup['low_tail'] = abs(df_setup['lowprice'] - df_setup[['openprice', 'closeprice']].min(axis=1))

# Calculate tail to body ratio
df_setup['high_tail_ratio'] = df_setup['high_tail'] / df_setup['body_size']
df_setup['low_tail_ratio'] = df_setup['low_tail'] / df_setup['body_size']

# Calculate the max highprice for the next hours
df_setup['next_1h_max_chg'] = (df_setup['highprice'].shift(-4).rolling(window=4).max() - df_setup['openprice']) / df_setup['openprice'] *100
df_setup['next_2h_max_chg'] = (df_setup['highprice'].shift(-8).rolling(window=8).max() - df_setup['openprice']) / df_setup['openprice'] *100
df_setup['next_3h_max_chg'] = (df_setup['highprice'].shift(-12).rolling(window=12).max() - df_setup['openprice']) / df_setup['openprice'] *100
df_setup['next_4h_max_chg'] = (df_setup['highprice'].shift(-16).rolling(window=16).max() - df_setup['openprice']) / df_setup['openprice'] *100

# Calculate the min lowprice for the next hours
df_setup['next_1h_min_chg'] = (df_setup['lowprice'].shift(-4).rolling(window=4).min() - df_setup['openprice']) / df_setup['openprice'] *100
df_setup['next_2h_min_chg'] = (df_setup['lowprice'].shift(-8).rolling(window=8).min() - df_setup['openprice']) / df_setup['openprice'] *100
df_setup['next_3h_min_chg'] = (df_setup['lowprice'].shift(-12).rolling(window=12).min() - df_setup['openprice']) / df_setup['openprice'] *100
df_setup['next_4h_min_chg'] = (df_setup['lowprice'].shift(-16).rolling(window=16).min() - df_setup['openprice']) / df_setup['openprice'] *100

df_setup.dropna(inplace=True)

df_setup

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,high_tail_ratio,low_tail_ratio,next_1h_max_chg,next_2h_max_chg,next_3h_max_chg,next_4h_max_chg,next_1h_min_chg,next_2h_min_chg,next_3h_min_chg,next_4h_min_chg
15,1683143100000,1.3239,1.3315,1.3088,1.3179,3643283.4,4.807001e+06,17157,1880578.8,2.482392e+06,...,1.266667,1.516667,3.852255,4.116625,6.503512,6.692348,-1.450261,-1.450261,-1.450261,-1.450261
16,1683144000000,1.3178,1.3370,1.3047,1.3310,3605087.5,4.768979e+06,16964,1859842.1,2.462070e+06,...,0.454545,0.992424,4.598573,4.598573,6.996509,10.563060,0.531188,0.531188,0.531188,0.531188
17,1683144900000,1.3310,1.3600,1.3248,1.3482,6539643.5,8.812810e+06,26233,3582157.7,4.828627e+06,...,0.686047,0.360465,3.561232,5.935387,5.935387,10.428249,0.300526,0.300526,0.300526,0.300526
18,1683145800000,1.3476,1.3620,1.3350,1.3550,5554826.3,7.496456e+06,26168,2922951.2,3.946119e+06,...,0.945946,1.702703,2.285545,4.630454,4.815969,10.789552,-0.563966,-0.563966,-0.563966,-0.563966
19,1683146700000,1.3552,1.3749,1.3551,1.3713,4662833.4,6.366378e+06,21508,2636401.4,3.601037e+06,...,0.223602,0.006211,1.711924,4.043684,4.228158,10.175620,-1.121606,-1.121606,-1.121606,-1.121606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58307,1735605900000,4.1300,4.1450,4.0701,4.0757,3469265.5,1.421439e+07,37657,1320962.7,5.410177e+06,...,0.276243,0.103131,-0.847458,-0.803874,-0.479419,-0.135593,-2.092010,-2.092010,-2.092010,-2.092010
58308,1735606800000,4.0760,4.0823,4.0436,4.0566,2775273.0,1.127204e+07,37278,1323282.8,5.377171e+06,...,0.324742,0.670103,0.466143,0.510304,1.187439,1.187439,-0.753189,-0.753189,-0.753189,-0.753189
58309,1735607700000,4.0565,4.0820,4.0453,4.0689,2078719.4,8.458241e+06,26775,1142574.5,4.648512e+06,...,1.056452,0.903226,0.949094,1.281893,1.673857,1.673857,0.216936,0.216936,0.216936,0.125724
58310,1735608600000,4.0688,4.0894,4.0658,4.0848,1311306.3,5.348425e+06,22300,753842.4,3.074772e+06,...,0.287500,0.187500,0.688164,1.017499,1.366496,1.528706,-0.086020,-0.086020,-0.086020,-0.176956


# Filtering

In [36]:
# Tail >= 1.5, bear RHP 
df_setup_ratio_15 = df_setup[(df_setup['high_tail_ratio'] >= 1.5) & (df_setup['bull'] == 0)][['opentime', 'openprice', 'closeprice', 'min_1h_chg', 'min_2h_chg', 'min_3h_chg', 'min_4h_chg']]

df_setup_ratio_15

,opentime,openprice,closeprice,min_1h_chg,min_2h_chg,min_3h_chg,min_4h_chg
9,1683137700000,1.3410,1.3382,-4.675669,-4.675669,-4.675669,-4.675669
20,1683147600000,1.3713,1.3670,-2.335821,-2.335821,-2.335821,-2.335821
25,1683152100000,1.3735,1.3700,-1.148833,-1.148833,-1.148833,-1.148833
30,1683156600000,1.3858,1.3773,-1.479203,-1.479203,-1.479203,-1.479203
51,1683175500000,1.4128,1.4125,-1.977768,-4.651852,-4.651852,-4.651852
...,...,...,...,...,...,...,...
58313,1735611300000,4.0838,4.0805,-0.440247,-0.440247,-0.546583,-0.546583
58314,1735612200000,4.0806,4.0723,-0.331932,-0.331932,-0.467796,-0.467796
58316,1735614000000,4.0797,4.0760,-0.309803,-0.445637,-0.445637,-0.445637
58324,1735621200000,4.0829,4.0810,-0.472476,-0.472476,-0.472476,-0.472476


In [25]:
df_setup_ratio_15.columns

Index(['opentime', 'openprice', 'highprice', 'lowprice', 'closeprice',
       'volume', 'quotevolume', 'trades', 'taker_buy_volume',
       'taker_buy_quote', 'bull', 'body_size', 'high_tail', 'low_tail',
       'high_tail_ratio', 'low_tail_ratio', 'max_1h_norm', 'min_1h_norm',
       'max_2h_norm', 'min_2h_norm', 'max_3h_norm', 'min_3h_norm',
       'max_4h_norm', 'min_4h_norm'],
      dtype='object')

In [24]:
df_setup_ratio_15.describe()

/home/ubuntu/Rheza/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/ubuntu/Rheza/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,high_tail_ratio,low_tail_ratio,max_1h_norm,min_1h_norm,max_2h_norm,min_2h_norm,max_3h_norm,min_3h_norm,max_4h_norm,min_4h_norm
count,7.304000e+03,7304.000000,7304.000000,7304.000000,7304.000000,7.304000e+03,7.304000e+03,7304.000000,7.304000e+03,7.304000e+03,...,7304.000000,7299.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000
mean,1.709273e+12,1.232568,1.237408,1.228034,1.231198,3.060937e+06,3.861141e+06,10267.652382,1.494293e+06,1.886021e+06,...,inf,inf,0.745591,-0.082310,1.111825,-0.434328,1.392858,-0.707332,1.621870,-0.947178
std,1.518547e+10,0.920494,0.924823,0.916587,0.919291,3.694078e+06,5.753968e+06,11809.299162,1.834572e+06,2.828539e+06,...,NaN,NaN,0.704329,0.512345,1.110029,0.917683,1.401056,1.239467,1.618318,1.470838
min,1.683138e+12,0.364300,0.364800,0.363900,0.364200,1.252340e+05,9.333638e+04,490.000000,4.385540e+04,2.315318e+04,...,1.500000,0.000000,0.024462,-7.732733,0.024462,-15.822570,0.024462,-19.785641,0.024462,-19.785641
25%,1.695996e+12,0.640900,0.644175,0.639000,0.640475,1.056510e+06,8.618895e+05,3015.000000,5.091506e+05,4.138782e+05,...,2.181235,0.916667,0.319387,-0.259660,0.410459,-0.740514,0.473978,-1.109175,0.536493,-1.446916
50%,1.708898e+12,0.910150,0.913750,0.906650,0.908750,1.911312e+06,1.880319e+06,6248.000000,9.195318e+05,9.140296e+05,...,3.603571,2.604167,0.535864,0.041524,0.750243,-0.179106,0.946174,-0.364282,1.106545,-0.536556
75%,1.722625e+12,1.544325,1.550100,1.538950,1.542825,3.658955e+06,4.622681e+06,13301.500000,1.780842e+06,2.244805e+06,...,8.750000,8.000000,0.921592,0.207423,1.394571,0.122253,1.790401,0.064722,2.119543,0.015393
max,1.735624e+12,4.896800,4.959100,4.886400,4.891000,5.892245e+07,1.004974e+08,157954.000000,2.715860e+07,4.638149e+07,...,inf,inf,8.256439,1.981982,12.621263,1.981982,16.633921,1.981982,18.845055,1.727758


In [13]:
df_setup[df_setup['high_tail_ratio'] >= 5]

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,bull,body_size,high_tail,low_tail,high_tail_ratio,low_tail_ratio
9,1683137700000,1.3410,1.3550,1.3212,1.3382,6813472.4,9.098928e+06,30449,3485406.3,4.656880e+06,0,0.0028,0.0140,0.0170,5.000000,6.071429
14,1683142200000,1.3228,1.3408,1.3011,1.3236,5829099.9,7.714365e+06,24920,2879720.2,3.814214e+06,1,0.0008,0.0172,0.0217,21.500000,27.125000
25,1683152100000,1.3735,1.4100,1.3661,1.3700,9393635.3,1.301280e+07,40419,5381982.7,7.459925e+06,0,0.0035,0.0365,0.0039,10.428571,1.114286
36,1683162000000,1.4652,1.4819,1.4611,1.4670,6627612.7,9.767116e+06,28424,3317252.4,4.889896e+06,1,0.0018,0.0149,0.0041,8.277778,2.277778
51,1683175500000,1.4128,1.4147,1.4025,1.4125,3359611.6,4.736749e+06,12088,1637805.5,2.309516e+06,0,0.0003,0.0019,0.0100,6.333333,33.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58276,1735578000000,4.0948,4.1268,4.0775,4.0986,4480303.4,1.839814e+07,44861,2020927.7,8.298244e+06,1,0.0038,0.0282,0.0173,7.421053,4.552632
58285,1735586100000,4.2982,4.3264,4.2893,4.3026,4614673.9,1.988017e+07,41022,2001781.9,8.624734e+06,1,0.0044,0.0238,0.0089,5.409091,2.022727
58290,1735590600000,4.2826,4.3032,4.2632,4.2849,3822970.9,1.636942e+07,36536,2245400.4,9.617320e+06,1,0.0023,0.0183,0.0194,7.956522,8.434783
58324,1735621200000,4.0829,4.0953,4.0738,4.0810,854988.6,3.493640e+06,13288,392507.9,1.603318e+06,0,0.0019,0.0124,0.0072,6.526316,3.789474
